In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import spatialdata as sd

from multiplex_pipeline.core_segmentation.controller import SegmentationController
from multiplex_pipeline.core_segmentation.segmenters import CellposeSegmenter, InstansegSegmenter
from multiplex_pipeline.core_preprocessing.controller import PreSegmentationProcessor

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [3]:
sdata_path = r'D:\multiplex_pipeline - Copy\temp\Core_005.zarr' 

In [4]:
sdata_org = sd.read_zarr(sdata_path)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


In [5]:
sdata_org

SpatialData object, with associated Zarr store: D:\multiplex_pipeline - Copy\temp\Core_005.zarr
└── Images
      ├── 'CD8a': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
      ├── 'CD11C': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
      ├── 'CD44': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
      ├── 'CD45': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
      ├── 'DAPI': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
      ├── 'HLA1': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
      ├── 'NaKATPase': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
      ├── 'pCK26': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
      └── 'pS6': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
with coordinate systems:
    ▸ 'global', with elements:
        CD8a (Images), CD11C (Images), CD44 (Images), CD45 (Images), DAPI (Images), HLA1 (Ima

## Cellpose nuclear segmentation

In [6]:
cellpose_segmenter = CellposeSegmenter(model_type = 'cyto2', diameter = 40)
segmentation_controller = SegmentationController(cellpose_segmenter, resolution_level = 0)

In [7]:
segmentation_controller.segment_spatial_data(sdata_path, channels = ['DAPI'], mask_name = "cellpose_nucleus")

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\cellpose-env\Lib\site-packages\zarr\creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


In [8]:
# check content of the SpatialData object
sdata = sd.read_zarr(sdata_path)
sdata

version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: \\share.files.pitt.edu\HSIT-Stallaert-Lab\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_015.zarr
├── Images
│     ├── 'CD3': DataTree[cyx] (1, 5824, 5824), (1, 2912, 2912), (1, 1456, 1456)
│     └── 'DAPI': DataTree[cyx] (1, 5824, 5824), (1, 2912, 2912), (1, 1456, 1456)
└── Labels
      └── 'cellpose_nucleus': DataTree[yx] (5824, 5824), (2912, 2912), (1456, 1456)
with coordinate systems:
    ▸ 'global', with elements:
        CD3 (Images), DAPI (Images), cellpose_nucleus (Labels)

## Instanseg for nucleus and cell segmentation using multiple channels

In [6]:
channels = ['DAPI','HLA1','CD8a','CD44','NaKATPase','CD11C','CD45','pCK26','pS6']

In [7]:
# get 8 bit normalized images to use with instanseg

preseg_processor = PreSegmentationProcessor(normalize = [1, 99.5])
preseg_processor.run(sdata_org, channels = channels)

2025-07-08 22:06:45.921 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:80 - Applied normalization with percentiles [1, 99.5].
2025-07-08 22:06:49.910 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:80 - Applied normalization with percentiles [1, 99.5].
2025-07-08 22:06:53.825 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:80 - Applied normalization with percentiles [1, 99.5].
2025-07-08 22:06:57.691 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:80 - Applied normalization with percentiles [1, 99.5].
2025-07-08 22:07:01.598 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:80 - Applied normalization with percentiles [1, 99.5].
2025-07-08 22:07:05.831 | INFO     | multiplex_pipeline.core_preprocessing.controller:run_normalize:80 - Applied normalization with percentiles [1, 99.5].
2025-07-08 22:07:09.866 | INFO     | multiplex_pipeline.core_preproces

In [14]:
# refresh the object
sdata_org = sd.read_zarr(sdata_path)
sdata_org

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: D:\multiplex_pipeline - Copy\temp\Core_005.zarr
├── Images
│     ├── 'CD8a': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'CD11C': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'CD44': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'CD45': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'DAPI': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'HLA1': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'NaKATPase': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'pCK26': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     └── 'pS6': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
└── Labels
      ├── 'instanseg_cell': DataTree[yx] (5760, 5824), (2880, 2912), (1440, 1456)
      └── 'instanseg_nucleus': DataTree[yx] (5760, 5824)

In [9]:
test_segmenter = InstansegSegmenter(model_type = 'fluorescence_nuclei_and_cells', pixel_size = 0.3, resolve_cell_and_nucleus = True, cleanup_fragments = True)
segmentation_controller = SegmentationController(test_segmenter, resolution_level = 0)

Model fluorescence_nuclei_and_cells version 0.1.0 already downloaded in c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\instanseg\utils\../bioimageio_models/, loading
Requesting default device: cuda


2025-07-08 22:09:50.939 | INFO     | multiplex_pipeline.core_segmentation.controller:__init__:19 - Initialized SegmentationController with segmenter=<multiplex_pipeline.core_segmentation.segmenters.InstansegSegmenter object at 0x0000025A49037110>, resolution_level=0, pyramid_levels=3, downscale=2


In [10]:
channels = [f'{ch}_preseg' for ch in channels]

In [11]:
segmentation_controller.segment_spatial_data(sdata_path, channels = channels, mask_name = ['instanseg_nucleus', 'instanseg_cell'])

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requeste

In [12]:
for ch in channels:
    del sdata_org[ch]
    sdata_org.delete_element_from_disk(ch)

In [15]:
# refresh the object
sdata_org = sd.read_zarr(sdata_path)
sdata_org

version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: D:\multiplex_pipeline - Copy\temp\Core_005.zarr
├── Images
│     ├── 'CD8a': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'CD11C': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'CD44': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'CD45': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'DAPI': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'HLA1': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'NaKATPase': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     ├── 'pCK26': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
│     └── 'pS6': DataTree[cyx] (1, 5760, 5824), (1, 2880, 2912), (1, 1440, 1456)
└── Labels
      ├── 'instanseg_cell': DataTree[yx] (5760, 5824), (2880, 2912), (1440, 1456)
      └── 'instanseg_nucleus': DataTree[yx] (5760, 5824)

In [ ]:
from napari_spatialdata import Interactive

interactive = Interactive(sdata_org)
interactive.run()

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
2025-07-08 22:28:35.570 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-07-08 22:28:35.578 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-07-08 22:28:44.843 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-07-08 22:28:44.853 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-07-08 22:29:10.423 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-07-08 22:29:10.433 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-07-08 22:29:59.925 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-07-08 22:29:59.934 | DEBUG    | napari_spatialdata.